In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

# 1. Loading data and data inspection

## 1.1 General overview

### 1.1.1. File structure check

In [2]:
data_folder = "data"

click_data = pd.read_csv(f"{data_folder}/Clicks.csv", parse_dates=["click_timestamp","registration_date"])
offers_data = pd.read_csv(f"{data_folder}/Offers.csv")
postbacks_data = pd.read_csv(f"{data_folder}/Postbacks.csv", parse_dates=["postback_timestamp"])
source_data = pd.read_csv(f"{data_folder}/Source.csv")

display(click_data.head())
display(offers_data.head())
display(postbacks_data.head())
display(source_data.head())

,user_id,click_id,offer_id,source_id,partner,browser,placement,device_type,geo,os,click_number,is_split_offer,click_timestamp,registration_date
0,144264257599918,200986338821580,1.083392e+14,30813769543521,2.335252e+14,Chrome Mobile,top,MOBILE,FR,Android,591,0,2024-10-01 20:48:16,2024-01-04
1,54289169784097,80191043633935,2.055478e+14,205473966316305,1.052342e+14,Wave,other,DESKTOP,US,Windows,21,1,2024-10-01 21:03:28,2023-12-09
2,55499320366337,171674324918689,1.422237e+14,28948219646125,2.894822e+13,Chrome Mobile,interstitial,MOBILE,LV,Android,18,0,2024-10-01 20:48:57,2024-09-23
3,93900209625875,117345394691017,2.523488e+14,75520303016228,7.552030e+13,Chromium Mobile,bottom,MOBILE,AT,Android,2540,0,2024-10-01 20:49:00,2024-09-30
4,37122870576500,58465048649323,5.778412e+13,17526471573325,1.053868e+14,Chromium Mobile,interstitial,MOBILE,ES,Android,6,0,2024-10-01 20:49:41,2024-10-01


,offer_id,payout_type,affiliate_network,daily_cap_amount,total_cap_amount,is_backfill
0,142913016698785,PPL,150066265098611,50.0,NaN,0
1,45516139449060,PPL,150066265098611,50.0,NaN,0
2,106876863872785,PPS,150066265098611,40.0,NaN,0
3,16479728416122,PPL,150066265098611,5.0,NaN,0
4,52028205626427,PPL,150066265098611,30.0,NaN,0


,click_id,offer_id,revenue,postback_timestamp
0,1944536109713,30635717719102,0.03,2024-10-18 17:09:33
1,64822103575831,230701590537978,0.09,2024-10-13 11:50:55
2,200444034598409,274707435099920,0.05,2024-10-05 12:05:29
3,99175274672156,71765294892525,0.09,2024-10-19 09:50:16
4,88772326326384,101043498568515,0.05,2024-10-18 05:34:52


,source_id,network
0,279593647644957,263971225698170
1,27794914753301,87332366395962
2,109647236483577,230005278200733
3,51820170826895,124393588045669
4,151036457625833,43549091994663


In [3]:
print(f"Clicks: {len(click_data)} записів")
print(f"Offers: {len(offers_data)} записів") 
print(f"Postbacks: {len(postbacks_data)} записів")
print(f"Sources: {len(source_data)} записів")

Clicks: 6060765 записів
Offers: 74547 записів
Postbacks: 1829668 записів
Sources: 36914 записів


In [4]:
print("All countries:")
for country in click_data['geo'].unique():
    print(country)

All countries:
FR
US
LV
AT
ES
GB
AR
CA
PL
DE
DK
BR
CO
NO
IT
DO
SE
AU
IN
CL
ID
RO
CZ
GR
IL
MX
FI
TR
NL
LT
BH
IR
PK
YE
IE
ZA
UG
RW
TD
JO
VE
RU
PH
BD
PT
EE
SK
BE
ET
LY
LB
BG
SL
UY
RS
MA
JP
NZ
FJ
HK
GP
GM
KR
PR
CG
PY
PE
CY
DZ
UA
SA
PG
ZM
CH
SD
HU
CR
NC
SB
MN
SG
LU
CD
MG
MW
HR
AF
AE
ML
NI
MT
GN
EC
LK
TC
IQ
PF
AM
NP
EG
SO
CU
QA
MM
MY
KM
ZW
SY
BA
CM
BI
UZ
GE
PS
IS
TZ
KW
TN
GA
CI
MR
MD
TM
SN
BW
TG
VI
CN
TL
VU
KE
HT
AL
SC
YT
MK
KG
GY
NE
JM
KZ
TH
GQ
**
OM
AW
SI
TJ
AO
KI
TT
SZ
VG
BO
KH
GT
TW
DJ
ME
HN
RE
LS
LR
SS
AS
BN
MU
BY
ST
BF
GH
CW
IM
GL
MV
MQ
DM
nan
NG
PA
CV
AI
LA
LC
BJ
SV
WS
GW
BB
MZ
FO
CF
GD
PM
SM
LI
BZ
AD
TO
VN
SR
BS
AG
CK
GF
BT
JE
AZ
BQ
BL
MC
GI
AX
PW
SX
KY
MO
GU
KN
VC
GG
FM
WF
MP
MF
BM
EU
MH
NR


### 1.1.2. Data quality check

In [5]:
for df_name, df in [('clicks', click_data), ('offers', offers_data), ('postbacks', postbacks_data)]:
    print(f"\n{df_name} missing values:")
    print(df.isnull().sum())

print(f"Click duplicates: {click_data.duplicated().sum()}")
print(f"Postback duplicates: {postbacks_data.duplicated().sum()}")

print(f"Unique offers in clicks: {click_data['offer_id'].nunique()}")
print(f"Unique offers in offers_data: {offers_data['offer_id'].nunique()}")
print(f"Unique click_ids in postbacks: {postbacks_data['click_id'].nunique()}")


clicks missing values:
user_id                0
click_id               0
offer_id             829
source_id              0
partner               15
browser                5
placement              0
device_type            0
geo                  127
os                     0
click_number           0
is_split_offer         0
click_timestamp        0
registration_date     17
dtype: int64

offers missing values:
offer_id                 0
payout_type              0
affiliate_network        0
daily_cap_amount     10422
total_cap_amount     15620
is_backfill              0
dtype: int64

postbacks missing values:
click_id              0
offer_id              0
revenue               0
postback_timestamp    0
dtype: int64
Click duplicates: 0
Postback duplicates: 0
Unique offers in clicks: 4979
Unique offers in offers_data: 74547
Unique click_ids in postbacks: 1664886


## 1.2. Basic statistics

### 1.2.1. Clicks

In [7]:
# Часовий діапазон
click_data['click_timestamp'] = pd.to_datetime(click_data['click_timestamp'])
print(f"Дата від: {click_data['click_timestamp'].min()}")
print(f"Дата до: {click_data['click_timestamp'].max()}")
print(f"Днів даних: {(click_data['click_timestamp'].max() - click_data['click_timestamp'].min()).days}\n")

# Розподіл по офферах
offer_clicks = click_data.groupby('offer_id').size().sort_values(ascending=False)
print(f"Топ-10 офферів по кількості кліків:")
print(offer_clicks.head(10))

# Розподіл по геолокації
geo_dist = click_data['geo'].value_counts()
print(f"\nРозподіл по країнах:")
print(geo_dist.head(10))

# Розподіл по пристроях
device_dist = click_data['device_type'].value_counts()
print(f"\nРозподіл по пристроях:")
print(device_dist)

# is_split_offer розподіл
split_dist = click_data['is_split_offer'].value_counts()
print(f"\nРозподіл is_split_offer:")
print(split_dist)

Дата від: 2024-10-01 00:00:03
Дата до: 2024-10-31 23:59:59
Днів даних: 30

Топ-10 офферів по кількості кліків:
offer_id
5.675980e+13    91074
1.422237e+14    66217
2.151599e+14    65036
1.353057e+14    62547
8.436152e+13    58377
1.677992e+14    54094
1.297073e+14    53655
1.761090e+14    52861
4.479621e+13    52776
1.712228e+14    49800
dtype: int64

Розподіл по країнах:
geo
US    2394295
DE     713725
FR     483941
ES     329700
GB     279837
CA     214546
PL     203436
IT     156974
MX     139955
AU     131846
Name: count, dtype: int64

Розподіл по пристроях:
device_type
MOBILE      5666915
DESKTOP      212179
TABLET       176880
SMART_TV       4786
Mobile            5
Name: count, dtype: int64

Розподіл is_split_offer:
is_split_offer
0    4841087
1    1219678
Name: count, dtype: int64


### 1.2.2. Offers

In [8]:
# Типи оплати
payout_types = offers_data['payout_type'].value_counts()
print(f"Типи оплати офферів:")
print(payout_types)

# Кепи
print(f"\nСтатистика daily_cap_amount:")
print(offers_data['daily_cap_amount'].describe())

print(f"\nСтатистика total_cap_amount:")
print(offers_data['total_cap_amount'].describe())

# Backfill офери
backfill_dist = offers_data['is_backfill'].value_counts()
print(f"\nBackfill офери:")
print(backfill_dist)

# Affiliate networks
networks = offers_data['affiliate_network'].value_counts()
print(f"\nТоп affiliate networks:")
print(networks.head(10))

Типи оплати офферів:
payout_type
PPC    62969
PPL     9293
PPS     2285
Name: count, dtype: int64

Статистика daily_cap_amount:
count    6.412500e+04
mean     2.121705e+06
std      3.616487e+07
min      0.000000e+00
25%      1.000000e+02
50%      5.000000e+02
75%      1.000000e+04
max      2.147484e+09
Name: daily_cap_amount, dtype: float64

Статистика total_cap_amount:
count    5.892700e+04
mean     1.199525e+08
std      3.743294e+08
min      1.000000e+00
25%      1.000000e+04
50%      1.000000e+05
75%      2.000000e+07
max      2.147484e+09
Name: total_cap_amount, dtype: float64

Backfill офери:
is_backfill
0    62736
1    11811
Name: count, dtype: int64

Топ affiliate networks:
affiliate_network
138118447793570    6226
199617847123945    4310
154133161358371    3541
9537682357693      2450
81174045035506     1978
140419050228329    1854
91073527378745     1534
260055600451761    1227
74503830903853     1211
237693237470769    1174
Name: count, dtype: int64


### 1.2.3. Postbacks

In [10]:
# Часовий діапазон постбеків
postbacks_data['postback_timestamp'] = pd.to_datetime(postbacks_data['postback_timestamp'])
print(f"Постбеки від: {postbacks_data['postback_timestamp'].min()}")
print(f"Постбеки до: {postbacks_data['postback_timestamp'].max()}")

# Статистика revenue
print(f"\nСтатистика revenue:")
print(postbacks_data['revenue'].describe())

# Розподіл revenue
revenue_dist = postbacks_data['revenue'].value_counts().sort_index(ascending=False)
print(f"\nТоп значення revenue:")
print(revenue_dist.head(10))

Постбеки від: 2024-10-01 00:00:05
Постбеки до: 2024-10-31 23:59:57

Статистика revenue:
count    1.829668e+06
mean     2.653751e-01
std      2.499372e+00
min      0.000000e+00
25%      2.000000e-02
50%      4.000000e-02
75%      8.000000e-02
max      2.640000e+02
Name: revenue, dtype: float64

Топ значення revenue:
revenue
264.0    19
209.0     1
187.0     3
176.0     1
165.0     5
159.5     7
154.0     2
148.5     1
137.5     4
132.0     2
Name: count, dtype: int64


# 2. Conversions derivation

In [11]:
clicks_with_postbacks = click_data.merge(
    postbacks_data[['click_id', 'offer_id', 'revenue', 'postback_timestamp']], 
    on=['click_id', 'offer_id'], 
    how='left'
)

print(f"Кліків всього: {len(click_data)}")
print(f"Кліків з постбеками: {clicks_with_postbacks['revenue'].notna().sum()}")
print(f"Conversion rate загальний: {clicks_with_postbacks['revenue'].notna().mean():.3f}")

Кліків всього: 6060765
Кліків з постбеками: 1447115
Conversion rate загальний: 0.239


In [12]:
# Додати дані офферів
full_data = clicks_with_postbacks.merge(
    offers_data[['offer_id', 'payout_type', 'affiliate_network', 'is_backfill']], 
    on='offer_id', 
    how='left'
)

# Додати дані джерел
full_data = full_data.merge(
    source_data, 
    on='source_id', 
    how='left'
)

print(f"Фінальна таблиця: {len(full_data)} записів")

Фінальна таблиця: 6060765 записів


In [13]:
# Тільки для кліків з постбеками
conversions = full_data[full_data['revenue'].notna()].copy()
conversions['time_to_conversion'] = (
    conversions['postback_timestamp'] - conversions['click_timestamp']
).dt.total_seconds() / 3600  # в годинах

print(f"Середній час до конверсії: {conversions['time_to_conversion'].mean():.2f} годин")
print(f"Медіана часу до конверсії: {conversions['time_to_conversion'].median():.2f} годин")

# По типах оплати
time_by_payout = conversions.groupby('payout_type')['time_to_conversion'].agg(['mean', 'median', 'count'])
print(f"\nЧас до конверсії по типах оплати:")
print(time_by_payout)

Середній час до конверсії: 0.10 годин
Медіана часу до конверсії: 0.00 годин

Час до конверсії по типах оплати:
                 mean    median    count
payout_type                             
PPC          0.000551  0.000556  1330635
PPL          1.614778  0.100278    78612
PPS          0.529916  0.235833    37868


In [14]:
# Функція для розрахунку CR
def calculate_cr(data, group_columns):
    grouped = data.groupby(group_columns).agg({
        'click_id': 'count',
        'revenue': lambda x: x.notna().sum()
    }).rename(columns={'click_id': 'clicks', 'revenue': 'conversions'})
    grouped['cr'] = grouped['conversions'] / grouped['clicks']
    grouped['avg_revenue'] = data.groupby(group_columns)['revenue'].mean()
    return grouped.sort_values('clicks', ascending=False)

# CR по офферах
cr_by_offer = calculate_cr(full_data, ['offer_id'])
print("Топ-10 офферів по кількості кліків:")
print(cr_by_offer.head(10))

# CR по геолокації
cr_by_geo = calculate_cr(full_data, ['geo'])
print("\nCR по геолокації:")
print(cr_by_geo.head(10))

# CR по типу пристрою
cr_by_device = calculate_cr(full_data, ['device_type'])
print("\nCR по типу пристрою:")
print(cr_by_device)

# CR по комбінації оффер + гео
cr_by_offer_geo = calculate_cr(full_data, ['offer_id', 'geo'])
print("\nТоп комбінації оффер + гео:")
print(cr_by_offer_geo.head(20))

Топ-10 офферів по кількості кліків:
              clicks  conversions        cr  avg_revenue
offer_id                                                
5.675980e+13   91074         2406  0.026418     0.957814
1.422237e+14   66217          156  0.002356     1.236474
2.151599e+14   65036          180  0.002768     1.455778
1.353057e+14   62547         5238  0.083745     1.803723
8.436152e+13   58377          792  0.013567     9.153068
1.677992e+14   54094         2992  0.055311     1.483824
1.297073e+14   53655         3176  0.059193     1.787846
1.761090e+14   52861         3719  0.070354     1.640979
4.479621e+13   52776          578  0.010952     5.014896
1.712228e+14   49800          689  0.013835     6.050479

CR по геолокації:
      clicks  conversions        cr  avg_revenue
geo                                             
US   2394295       533986  0.223024     0.436280
DE    713725       267032  0.374138     0.112360
FR    483941       221792  0.458304     0.040548
ES    329700    

In [15]:
# Середній revenue по офферах
revenue_by_offer = full_data[full_data['revenue'].notna()].groupby('offer_id').agg({
    'revenue': ['mean', 'median', 'std', 'count']
}).round(2)
revenue_by_offer.columns = ['avg_revenue', 'median_revenue', 'std_revenue', 'conversions']
print("Revenue по офферах:")
print(revenue_by_offer.sort_values('conversions', ascending=False).head(10))

# ERPC (Expected Revenue Per Click)
erpc_data = []
for (offer_id, geo), group in full_data.groupby(['offer_id', 'geo']):
    if len(group) >= 10:  # Мінімум 10 кліків для валідності
        clicks = len(group)
        conversions = group['revenue'].notna().sum()
        cr = conversions / clicks if clicks > 0 else 0
        avg_revenue = group['revenue'].mean() if conversions > 0 else 0
        erpc = cr * avg_revenue
        
        erpc_data.append({
            'offer_id': offer_id,
            'geo': geo,
            'clicks': clicks,
            'conversions': conversions,
            'cr': cr,
            'avg_revenue': avg_revenue,
            'erpc': erpc
        })

erpc_df = pd.DataFrame(erpc_data)
erpc_df = erpc_df.sort_values('erpc', ascending=False)
print("\nТоп ERPC по комбінації оффер + гео:")
print(erpc_df.head(20))

Revenue по офферах:
              avg_revenue  median_revenue  std_revenue  conversions
offer_id                                                           
1.465611e+14         0.06            0.06          0.0        37203
7.176529e+13         0.09            0.09          0.0        28589
1.290234e+14         0.05            0.05          0.0        20802
2.106779e+13         0.18            0.18          0.0        19126
1.898687e+14         0.03            0.03          0.0        18986
1.912342e+14         0.15            0.15          0.0        18088
2.282508e+14         0.06            0.06          0.0        18045
2.619288e+14         0.06            0.06          0.0        17765
4.536761e+13         0.03            0.03          0.0        15803
1.735216e+13         0.05            0.05          0.0        14198

Топ ERPC по комбінації оффер + гео:
          offer_id geo  clicks  conversions        cr  avg_revenue      erpc
953   4.635060e+13  US     154            5  0.032

# 3. Bussiness Logic

In [16]:
# Порівняння CR для різних значень is_split_offer
split_comparison = full_data.groupby('is_split_offer').agg({
    'click_id': 'count',
    'revenue': lambda x: x.notna().sum()
}).rename(columns={'click_id': 'clicks', 'revenue': 'conversions'})
split_comparison['cr'] = split_comparison['conversions'] / split_comparison['clicks']
split_comparison['avg_revenue'] = full_data.groupby('is_split_offer')['revenue'].mean()
split_comparison['erpc'] = split_comparison['cr'] * split_comparison['avg_revenue']

print("Порівняння is_split_offer:")
print(split_comparison)

# По офферах
split_by_offer = full_data.groupby(['offer_id', 'is_split_offer']).agg({
    'click_id': 'count',
    'revenue': lambda x: x.notna().sum()
}).rename(columns={'click_id': 'clicks', 'revenue': 'conversions'})
split_by_offer['cr'] = split_by_offer['conversions'] / split_by_offer['clicks']

# Офери з обома типами split
offers_both_split = split_by_offer.reset_index().groupby('offer_id')['is_split_offer'].nunique()
offers_with_both = offers_both_split[offers_both_split == 2].index
print(f"\nОфери з обома типами is_split_offer: {len(offers_with_both)}")

Порівняння is_split_offer:
                 clicks  conversions        cr  avg_revenue      erpc
is_split_offer                                                       
0               4841087      1373022  0.283619     0.219267  0.062188
1               1219678        74093  0.060748     0.771904  0.046892

Офери з обома типами is_split_offer: 1153


In [17]:
# Додати дату кліку
full_data['click_date'] = full_data['click_timestamp'].dt.date

# Кількість кліків по офферах по днях
daily_clicks = full_data.groupby(['offer_id', 'click_date']).size().reset_index(name='daily_clicks')

# Кількість конверсій по офферах по днях  
daily_conversions = full_data[full_data['revenue'].notna()].groupby(['offer_id', 'click_date']).size().reset_index(name='daily_conversions')

# З'єднати з інформацією про кепи
daily_stats = daily_clicks.merge(daily_conversions, on=['offer_id', 'click_date'], how='left').fillna(0)
daily_stats = daily_stats.merge(offers_data[['offer_id', 'daily_cap_amount']], on='offer_id')

# Розрахувати використання капа
daily_stats['cap_usage'] = daily_stats['daily_conversions'] / daily_stats['daily_cap_amount']
daily_stats['cap_exceeded'] = daily_stats['daily_conversions'] > daily_stats['daily_cap_amount']

print("Статистика використання капів:")
print(f"Днів з перевищенням капа: {daily_stats['cap_exceeded'].sum()}")
print(f"Середнє використання капа: {daily_stats['cap_usage'].mean():.3f}")
print(f"Офери що часто перевищують кап:")
cap_exceed_by_offer = daily_stats.groupby('offer_id')['cap_exceeded'].agg(['sum', 'count', 'mean']).sort_values('sum', ascending=False)
print(cap_exceed_by_offer.head(10))

Статистика використання капів:
Днів з перевищенням капа: 206
Середнє використання капа: 0.041
Офери що часто перевищують кап:
              sum  count      mean
offer_id                          
2.106779e+13   31     31  1.000000
1.912342e+14   31     31  1.000000
4.664586e+13   26     28  0.928571
2.712955e+14   23     31  0.741935
4.536761e+13   19     31  0.612903
9.192857e+13   17     31  0.548387
8.436152e+13   15     31  0.483871
2.084299e+14   12     22  0.545455
1.088912e+14    8     14  0.571429
2.569230e+14    6     24  0.250000


In [18]:
backfill_analysis = full_data.groupby('is_backfill').agg({
    'click_id': 'count',
    'revenue': lambda x: x.notna().sum()
}).rename(columns={'click_id': 'clicks', 'revenue': 'conversions'})
backfill_analysis['cr'] = backfill_analysis['conversions'] / backfill_analysis['clicks']
backfill_analysis['avg_revenue'] = full_data.groupby('is_backfill')['revenue'].mean()

print("Аналіз backfill офферів:")
print(backfill_analysis)

# Часовий розподіл backfill
full_data['hour'] = full_data['click_timestamp'].dt.hour
hourly_backfill = full_data.groupby(['hour', 'is_backfill']).size().unstack(fill_value=0)
hourly_backfill['backfill_ratio'] = hourly_backfill[1] / (hourly_backfill[0] + hourly_backfill[1])
print("\nВикористання backfill по годинах:")
print(hourly_backfill['backfill_ratio'].sort_values(ascending=False))

Аналіз backfill офферів:
              clicks  conversions        cr  avg_revenue
is_backfill                                             
0.0          3957941       879799  0.222287     0.363745
1.0          2101995       567316  0.269894     0.067385

Використання backfill по годинах:
hour
17    0.387997
18    0.385802
16    0.384636
11    0.384015
12    0.379763
20    0.377581
15    0.376563
23    0.375404
19    0.372769
13    0.370055
22    0.369213
14    0.368412
21    0.367719
10    0.362396
0     0.360578
9     0.346369
8     0.313506
1     0.306964
7     0.291889
2     0.284426
3     0.276456
6     0.276170
5     0.273272
4     0.263101
Name: backfill_ratio, dtype: float64
